In [1]:
import os
import csv
import json
import pickle
import requests
from time import sleep

Consumer Key	XZoHfEhHTquhRinLcNrS
Consumer Secret	QozVgMYFgZkYwcCdXNOhApHklUlVKUxe
Request Token URL	https://api.discogs.com/oauth/request_token
Authorize URL	https://www.discogs.com/oauth/authorize
Access Token URL	https://api.discogs.com/oauth/access_token

In [2]:
key = 'XZoHfEhHTquhRinLcNrS'
secret = 'QozVgMYFgZkYwcCdXNOhApHklUlVKUxe'

### Get release thumbs by artist (much faster)

In [57]:
release_info = {}

In [5]:
release_info = pickle.load(open('release_info.p', 'rb'))

In [10]:
artist_id = 61761 #1
save_every = 10
directory = 'thumbs'

In [11]:
releases = set([_[6:-4] for _ in os.listdir(directory)])

In [37]:
while True:
    print(len(releases), end='\r')
    if artist_id % save_every == 0:
        pickle.dump(release_info, open('release_info.p', 'wb'))

    while True:
        sleep(1)
        url = f'https://api.discogs.com/artists/{artist_id}/releases?key={key}&secret={secret}'
        try:
            r = requests.get(url, headers={'User-agent': 'FooBarApp/3.0'})
            if r.status_code == 200 or r.status_code == 404:
                break
        except KeyboardInterrupt:
            raise
        except Exception as e:
            print(e)
            continue
    response = json.loads(r.text)
    if 'message' in response and response['message'] == 'Artist not found.':
        artist_id += 1
        continue

    for release in response.get('releases', []):
        image_url = release.get('thumb', '')

        id = release['main_release'] if release['type'] == 'master' else release['id']
        if image_url != '' and str(id) not in releases:
            artist = release.get('artist', '')
            title = release.get('title', '')
            release_info[str(id)] = (artist, title, image_url)

            while True:
                try:
                    r = requests.get(image_url, headers={'User-agent': 'FooBarApp/3.0'})
                    if r.status_code == 200 or r.status_code == 404 or r.status_code == 400:
                        break
                except KeyboardInterrupt:
                    raise
                except Exception as e:
                    print(e)
                    sleep(1)
                    continue
            if r.status_code == 200:
                with open(f'{directory}/thumb_{id}.jpg', 'wb') as file:
                    file.write(r.content)
            releases.add(str(id))
                
    artist_id += 1

KeyboardInterrupt: 

In [38]:
pickle.dump(release_info, open('release_info.p', 'wb'))

In [39]:
artist_id

147561

In [45]:
jim = set(release_info.keys())

In [46]:
bob = set([_[6:-4] for _ in os.listdir('thumbs')])

In [47]:
bob - jim

{'1409421',
 '152475',
 '156105',
 '3293979',
 '5586348',
 '75165',
 '9179624',
 '993858'}

In [48]:
jim - bob

{'17135415', '1840167', '4359689'}

In [56]:
for release in bob - jim:
    sleep(1)
    url = f'https://api.discogs.com/releases/{release}?key={key}&secret={secret}'
    r = requests.get(url, headers={'User-agent': 'FooBarApp/3.0'})
    response = json.loads(r.text)
    image_url = response.get('thumb', '')
    artist = response.get('artist', '')
    title = response.get('title', '')
    if release not in release_info:
        print(artist, title, image_url)
        release_info[release] = (artist, title, image_url)

 Electricity / Jam Jam https://img.discogs.com/YaaYpcc7twiWpO0U8AdIGsqIFHw=/fit-in/150x150/filters:strip_icc():format(jpeg):mode_rgb():quality(40)/discogs-images/R-75165-1153075881.jpeg.jpg
 Late Night Extention E.P. https://img.discogs.com/OfODuYsC4z-fd4rMfq4KUqsb3kg=/fit-in/150x150/filters:strip_icc():format(jpeg):mode_rgb():quality(40)/discogs-images/R-156105-1263823506.jpeg.jpg
 Live From Paris https://img.discogs.com/qqA6l5vAW9KwXsYXHjGfZRKANEQ=/fit-in/150x150/filters:strip_icc():format(jpeg):mode_rgb():quality(40)/discogs-images/R-9179624-1476173102-9219.jpeg.jpg
 Rough Cut Vol. 1 https://img.discogs.com/HHfveQBhzsK3A5JV6Bzf7MJ4fYY=/fit-in/150x150/filters:strip_icc():format(jpeg):mode_rgb():quality(40)/discogs-images/R-993858-1181852795.jpeg.jpg
 EP https://img.discogs.com/VxkEfaaFmIyZZTKYhtb-ke_xL84=/fit-in/150x150/filters:strip_icc():format(jpeg):mode_rgb():quality(40)/discogs-images/R-3293979-1324352388.jpeg.jpg
 Dunkin' Disco Donuts - Vol. 1 https://img.discogs.com/b3Dt4G43hg

In [62]:
pickle.dump(release_info, open('release_info.p', 'wb'))

In [69]:
for release in jim - bob:
    sleep(1)
    print(release_info[release][2])
    r = requests.get(release_info[release][2], headers={'User-agent': 'FooBarApp/3.0'})
    if r.status_code == 200:
        with open(f'thumbs/thumb_{release}.jpg', 'wb') as file:
            file.write(r.content)
            print(release)

https://img.discogs.com/YVGbI_Y3Eyg-aAkcGpXFrGFK95E=/fit-in/150x150/filters:strip_icc():format(jpeg):mode_rgb():quality(40)/discogs-images/R-4359689-1362775655-1824.jpeg.jpg
4359689


KeyError: '1840167'

In [67]:
del release_info['1840167']

In [68]:
del release_info['17135415']